<a href="https://colab.research.google.com/github/ptsurko/AIML-notebooks/blob/master/IMDB_Reviews_classifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sys import getsizeof

# Kernels to follow:
# https://www.kaggle.com/irinaabdullaeva/imdb-reviews-with-keras

TensorFlow 2.x selected.


In [2]:
import os
import zipfile

os.environ['KAGGLE_USERNAME'] = "pavelkey" # username from the json file
os.environ['KAGGLE_KEY'] = "2d38d31c6d5bdc84afc023ad38f79088" # key from the json file

!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews # api copied from kaggle

zipref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r')
zipref.extractall()
zipref.close()

data = pd.read_csv('./IMDB Dataset.csv')

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
label_encoder = LabelEncoder()
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [0]:
tfdif = TfidfVectorizer(min_df=5, ngram_range=(1, 1), analyzer='word', dtype=np.float32)
features_sparse = tfdif.fit_transform(data['review'])
feature_names = np.array(tfdif.get_feature_names())
features = features_sparse.todense()

In [24]:
print('type of features_sparse :', type(features_sparse))
print('type of features :', type(features))
print('features.nbytes  : ', features.nbytes)
print('features.dtype   : ', features.dtype)
print('features.shape   : ', features.shape)

type of features_sparse : <class 'scipy.sparse.csr.csr_matrix'>
type of features : <class 'numpy.matrix'>
features.nbytes  :  7419800000
features.dtype   :  float32
features.shape   :  (50000, 37099)


### Selecting top features

In [25]:
selector = SelectKBest(k=20000)
top_features = selector.fit_transform(features_sparse, data['sentiment'])
top_feature_names = feature_names[selector.get_support()]

top_features_data = pd.DataFrame(top_features.toarray(), columns=top_feature_names)
top_features_data.head()

,00,000,001,007,01,03,10,100,1000,101,102,105,11,117,11th,123,128,13,134,13th,14th,15,150,1500,15th,1600,163,16s,1700,175,1794,17th,180,1830,1840,1861,1863,1881,1888,1891,...,zhivago,zhou,zhu,zhuravli,ziegfeld,ziegler,ziering,zigzag,zilch,zima,zimbalist,zimmer,zion,zip,zippo,zippy,zis,ziyi,zizek,zodiac,zoey,zombie,zombies,zombification,zombified,zomcom,zoned,zoo,zoom,zooms,zorro,zosch,zu,zucco,zudina,zula,zulu,zurer,zwart,émigré
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.055369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.202855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Visualizing top feature names

In [26]:
indices = np.argsort(selector.scores_)[::-1]
print('top 20 features: ', feature_names[indices][:20])

top 20 features:  ['bad' 'worst' 'great' 'waste' 'awful' 'no' 'excellent' 'even' 'terrible'
 'nothing' 'worse' 'boring' 'best' 'wonderful' 'poor' 'minutes' 'stupid'
 'horrible' 'just' 'and']


In [27]:
# Split data to target (y) and features (X)
X = top_features_data
y = np.array(data['sentiment'])

# # Here we split data to training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=13)
print("Train dataset shape: {0}, \nTest dataset shape: {1}".format(X_train.shape, X_test.shape))

Train dataset shape: (25000, 20000), 
Test dataset shape: (25000, 20000)


In [28]:
clf = LogisticRegression(random_state=13, solver='lbfgs').fit(X_train, y_train)
weights = clf.coef_.flatten()
print("score: ", clf.score(X_test, y_test))

score:  0.891


### Top positive features


In [29]:
top_weights = weights.argsort()[-10:][::-1]
top_weight_features = top_feature_names[top_weights]
pd.DataFrame([weights[top_weights]], columns=top_weight_features)

,great,excellent,best,wonderful,perfect,amazing,well,enjoyed,today,fun
0,7.974207,6.460998,5.915375,4.584863,4.526657,4.34986,4.103172,4.072997,4.05618,3.837827


### Top negative features

In [30]:
bottom_weights = weights.argsort()[:10]
bottom_weight_features = top_feature_names[bottom_weights]
pd.DataFrame([weights[bottom_weights]], columns=bottom_weight_features)

,worst,bad,awful,waste,boring,poor,nothing,terrible,dull,minutes
0,-9.24543,-8.148804,-6.866128,-6.512088,-6.387337,-5.631686,-5.525734,-5.24636,-4.309639,-4.269581
